In [1]:
import numpy as np
import pandas as pd
import pywt
import math
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
import sklearn.metrics as MK
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import signal
import random
from torch.optim.lr_scheduler import StepLR
from pylab import *
import heapq
import random
from sklearn.preprocessing import MinMaxScaler

In [7]:
def DOAS2(data_ys, data_bd, a=0, b=0, ac1=0, ac2=0):
    data_ys = data_ys
    data_bd = data_bd
    x1 = data_ys[:, 0].tolist() 
    y1 = data_ys[:, 1].tolist() 
    x2 = data_bd[:, 0].tolist()  
    y2 = data_bd[:, 1].tolist()  #
    xs_list = []
    lnxs_list = []
    i = 0
    for s_num in y1:
        k = s_num / y2[i]
        k1 = np.log(k)
        xs_list.append(k)
        lnxs_list.append(k1)
        i += 1
    x_jeiduan = x1[a:b]
    y_jieduan = xs_list[a:b]
    x_jeiduan2 = x1[b:(b + b - a)]
    y_jieduan2 = y_jieduan[::-1]
    ac1 = 0
    ac2 = 0
    x_jian = x_jeiduan[ac1:ac2 + 1]
    y_jian = y_jieduan[ac1:ac2 + 1]
    del x_jeiduan[ac1:ac2 + 1]
    del y_jieduan[ac1:ac2 + 1]
    z = np.polyfit(x_jeiduan, y_jieduan,5)  #多项式拟合
    p = np.poly1d(z)
    yvals = p(x2)
    doas_list = []
    i = 0
    for s_num in xs_list:
        k2 = s_num / yvals[i]
        k2 = np.log(k2)
        doas_list.append(k2)
        i += 1
    return doas_list[a:b]

In [ ]:
def DOAS3(x, y, js=6, ac1=0, ac2=0, ac3=0, ac4=0):
    x1 = np.array(x)
    y1 = np.array(y)
    x_filtered = np.delete(x1, np.r_[ac1:ac2 + 1, ac3:ac4 + 1])
    y_filtered = np.delete(y1, np.r_[ac1:ac2 + 1, ac3:ac4 + 1])
    z = np.polyfit(x_filtered, y_filtered, js) 
    p = np.poly1d(z) 
    y_fitted = p(x1)
    doas_list = y1 - y_fitted 
    return doas_list

In [295]:
q1 = 90
p1 = 115
qq1 = 30
pp1 = 50

In [203]:
bd = np.loadtxt(r"E:\data\acetone\7.27\bd\0000012.txt")
dc = np.loadtxt(r"E:\data\acetone\7.27\5ppm\0000015.txt")
ace_h = DOAS1(dc, bd, a=q1, b=p1, ac1=qq1, ac2=pp1)

In [343]:
bd = np.loadtxt(r"E:\data\acetone\6.29\bd.txt")
dc = np.loadtxt(r"E:\data\acetone\6.29\acetone\3000.txt")
ace_h,_,_ =DOAS123(dc, bd, a=q1, b=p1, ac1=qq1, ac2=pp1)
# ace_h =DOAS2(dc, bd, a=q1, b=p1, ac1=0, ac2=0)

In [318]:
bd = np.loadtxt(r"E:\data\acetone\7.11\bd\0000011.txt")
dc = np.loadtxt(r"E:\data\acetone\7.11\nh3\8067\0000023.txt")
# NH3_h = DOAS1(dc, bd, a=q1, b=p1, ac1=0, ac2=0)
NH3_h = DOAS2(dc, bd, a=q1, b=p1, ac1=0, ac2=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7956\2507448201.py:38: RuntimeWarning: invalid value encountered in log
  k2 = np.log(k2)


In [319]:
bd = np.loadtxt(r"E:\data\acetone\7.6\bd\0000018.txt")
dc = np.loadtxt(r"E:\data\acetone\7.6\o2\2.txt")
# O2_h,_,_ = DOAS223(dc, bd, a=q1, b=p1, ac1=0, ac2=0)
O2_h = DOAS2(dc, bd, a=q1, b=p1, ac1=0, ac2=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7956\2507448201.py:38: RuntimeWarning: invalid value encountered in log
  k2 = np.log(k2)


In [338]:
data_bd = np.loadtxt(r"E:\data\acetone\7.27\bd\0000002.txt")  # 读取背底数据
folder_path = r"E:\data\acetone\7.27\time"

In [344]:
mm = 0
u = []
v = []
po1 = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        daice = np.loadtxt(file_path)
        daice = np.array(daice)


        l1,xz1 = DOAS2(daice, data_bd, a=q1, b=p1,ac1=0, ac2=0,js =4,ac3 = 0,ac4 =0 )
        l1 = DOAS3(xz1,l1,q1=90, p1=115,js =4,qq1 = 30,pp1 =50 )

        po1.append(l1)
        ace = np.array(ace_h)
        o2 = np.array(O2_h)
        nh3 = np.array(NH3_h)
        l1 = np.array(l1)

        widths = np.arange(1, 151)

        A1 = []
        A2 = []
        A3 = []
        x1, y1, z1 = [], [], []
        x2, y2, z2 = [], [], []
        x3, y3, z3 = [], [], []
        ace, fre =  pywt.cwt(ace, widths, 'morl')
        l, fre =  pywt.cwt(l1, widths,'morl')
        o2, fre =  pywt.cwt(o2, widths, 'morl')
        nh3, fre = pywt.cwt(nh3, widths, 'morl')

        ace = ace.flatten(order='C')
        xl = l.flatten(order='C')
        o2 = o2.flatten(order='C')
        nh3 = nh3.flatten(order='C')


 ########################################O2############################################
        lamda = np.var((o2)) * 3 + np.mean(abs(o2))
        lll = 0
        for i in o2:
            if (i) > lamda:
                lll = lll + 1
        max_indices_o2 = sorted(range(len(o2)),
                                key=lambda i: o2[i],
                                reverse=True)[0:lll]
        for i in max_indices_o2:
            A1.append((xl[i]))
            x1.append((o2[i]))
            y1.append((nh3[i]))
            z1.append((ace[i]))
 ########################################NH3#############################################
        lamda = np.var((nh3)) * 3 + np.mean(abs(nh3))
        lll = 0
        for i in nh3:
            if (i) > lamda:
                lll = lll + 1
        max_indices_nh3 = sorted(range(len(nh3)),
                                 key=lambda i: nh3[i],
                                 reverse=True)[0:lll]
        for i in max_indices_nh3:
            A2.append((xl[i]))
            x2.append((o2[i]))
            y2.append((nh3[i]))
            z2.append((ace[i]))
 ########################################ACE#############################################
        h_max_number = []
        l_max_number = []
        ace1 = ace
        lamda = np.var((ace1)) * 3 + np.mean(abs(ace1))
        lll = 0
        for i in ace1:
            if (i) > lamda:
                lll = lll + 1
#         lll = 3000
        max_indices_ace = sorted(range(len(ace1)),
                                 key=lambda i: ace1[i],
                                 reverse=True)[0:lll]
        for i in max_indices_ace:
            A3.append((xl[i]))
            x3.append((o2[i]))
            y3.append((nh3[i]))
            z3.append((ace[i]))


        A_combined = np.concatenate((A1, A2, A3))
        x_combined = np.concatenate((x1, x2, x3))
        y_combined = np.concatenate((y1, y2, y3))
        z_combined = np.concatenate((z1, z2, z3))

        # 构造矩阵
        X = np.vstack(
            (x_combined, y_combined, z_combined)).T

        # 使用最小二乘法求解
        B = np.linalg.lstsq(X, A_combined, rcond=None)[0]

        a, b, c = B

        v.append(np.abs(c) * 4400)
        print(mm)
        print('o2', np.abs(a) * 4.98)
        print('nh3', np.abs(b) * 7940)
        print('ace', np.abs(c) * 4400)
        mm = mm + 1

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7956\2507448201.py:38: RuntimeWarning: invalid value encountered in log
  k2 = np.log(k2)


0
o2 1.9859954118728638
nh3 485.451963018626
ace 260.4069337248802
1
o2 1.9961223602294922
nh3 524.9543963000178
ace 260.4334428906441
2
o2 1.9982490539550781
nh3 618.957753829658
ace 281.55776113271713
3
o2 1.9977489709854126
nh3 576.51158503443
ace 260.0840851664543
4
o2 1.9869495630264282
nh3 529.4171660542488
ace 270.40401846170425
5
o2 1.9957903623580933
nh3 570.4091225937009
ace 254.8830583691597
6
o2 1.993835687637329
nh3 554.5402680113912
ace 270.36552876234055
7
o2 1.9905476570129395
nh3 614.5037592351437
ace 262.6295015215874
8
o2 1.987463116645813
nh3 574.7545696645975
ace 261.6386264562607
9
o2 2.0002198219299316
nh3 503.26619838550687
ace 251.58575177192688
10
o2 1.9916176795959473
nh3 587.9852866753936
ace 268.8337415456772
11
o2 2.012146234512329
nh3 565.8832438141108
ace 241.3911670446396
12
o2 1.9994152784347534
nh3 608.9251017048955
ace 247.92426824569702
13
o2 1.991333246231079
nh3 630.3402178511024
ace 266.6914612054825
14
o2 1.986354112625122
nh3 641.2352397814393


Traceback (most recent call last):
  File "D:\anaconda\Lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_7956\3491269958.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\anaconda\Lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\IPython\core\ultratb.py", line 1199, in structured_traceback
    return VerboseTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Li